<a href="https://colab.research.google.com/github/krishnannarayanaswamy/GenAI-Relevance-LLM-demo/blob/main/Central_Group_Hybrid_Search_with_Astra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cassandra-driver openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.9 MB/s eta 0:00:00


In [2]:
OPENAI_API_KEY=""
ASTRA_DB_SECURE_BUNDLE_PATH=""
ASTRA_DB_CLIENT_ID=''
ASTRA_DB_CLIENT_SECRET=''
ASTRA_DB_APPLICATION_TOKEN=''
ASTRA_DB_KEYSPACE=""
ASTRA_DB_ID=""
ASTRA_DB_REGION=""

In [45]:
# Astra Connection (Based on cqlsession, from cassio library)
import os
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

def getCQLSession(mode='astra_db'):
    print('Initializing CQL Session')
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_CLIENT_ID,
                ASTRA_DB_CLIENT_SECRET,
            ),
        )
        astraSession = cluster.connect()
        astraSession.row_factory = dict_factory
        print('Connected')
        return astraSession
    elif mode == 'local':
        cluster = Cluster()
        localSession = cluster.connect()
        return localSession
    else:
        raise ValueError('Unknown CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    return ASTRA_DB_KEYSPACE


In [46]:
# Open the cennection with Astra
session = getCQLSession()

Initializing CQL Session


ERROR:cassandra.connection:Closing connection <AsyncoreConnection(133190422492496) 49293bf1-9bb8-4c65-a060-cf566066cc00-us-east1.db.astra.datastax.com:29042:3a2e54a2-6c05-462c-91df-6b7b20f17a51> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Connected


In [5]:
# Commands needed on the script...
keyspace = ASTRA_DB_KEYSPACE
from cassandra.cqlengine.query import BatchStatement

cmd_create_products_table = f"""CREATE TABLE IF NOT EXISTS {keyspace}.products_cg_hybrid
(product_id int,
 brand text,
 product_name text,
 product_name_en text,
 short_description text,
 short_description_en text,
 long_description text,
 long_description_en text,
 product_link text,
 image_link text,
 saleprice text,
 product_categories text,
 openai_description_embedding_en vector<float, 1536>,
 openai_description_embedding_th vector<float, 1536>,
 PRIMARY KEY (product_id))"""

cmd_create_analyzer_index = f"""
CREATE CUSTOM INDEX IF NOT EXISTS brand_analyzer ON {keyspace}.products_cg_hybrid(brand)
USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {{ 'index_analyzer': 'thai'}};
"""

cmd_create_analyzer_second_index = f"""
CREATE CUSTOM INDEX IF NOT EXISTS categories_analyzer ON {keyspace}.products_cg_hybrid(product_categories)
USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {{ 'index_analyzer': '{{"tokenizer" : {{"name" : "thai"}},
	"filters" : [{{"name" : "porterstem"}}]}}'}};
"""

cmd_create_analyzer_third_index = f"""
CREATE CUSTOM INDEX IF NOT EXISTS name_analyzer ON {keyspace}.products_cg_hybrid(product_name)
USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {{ 'index_analyzer': '{{"tokenizer" : {{"name" : "thai"}},
	"filters" : [{{"name" : "porterstem"}}]}}'}};
"""

cmd_create_vector_index = f"""CREATE CUSTOM INDEX IF NOT EXISTS openai_en ON {keyspace}.products_cg_hybrid(openai_description_embedding_en)
USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'"""

cmd_create_vector_second_index = f"""CREATE CUSTOM INDEX IF NOT EXISTS openai_th ON {keyspace}.products_cg_hybrid(openai_description_embedding_th)
USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'"""


cmd_insert_product = f"""
                INSERT INTO {keyspace}.products_cg_hybrid
                (product_id, brand, product_name, product_name_en, short_description,short_description_en, long_description, long_description_en, product_link, image_link, saleprice, product_categories, openai_description_embedding_th, openai_description_embedding_en)
                VALUES (:product_id, :brand, :product_name, :product_name_en, :short_description, :short_description_en, :long_description, :long_description_en, :product_link, :image_link,:saleprice, :product_categories, :openai_description_embedding_th, :openai_description_embedding_en)
                """

cmd_select_thai_products = f"""
    SELECT product_id, product_name, short_description, long_description
    FROM {keyspace}.products_cg_hybrid
    WHERE product_name : keyword
    ORDER BY openai_description_embedding_th ANN OF :embedding
    LIMIT 10 """


cmd_select_english_products = f"""
    SELECT product_id, product_name_en, short_description_en, long_description_en
    FROM {keyspace}.products_cg_hybrid
    WHERE product_name : keyword
    ORDER BY openai_description_embedding_en ANN OF :embedding
    LIMIT 10 """

In [34]:
# More functions used on the code
import openai
openai.api_key = OPENAI_API_KEY
def generate_embedding(text):
    model = "text-embedding-ada-002"
    response = openai.Embedding.create(model=model, input=text)
    return response.data[0]['embedding']


In [39]:
# Create objects in DB
session.execute(cmd_create_products_table)
session.execute(cmd_create_analyzer_index)
session.execute(cmd_create_analyzer_second_index)
session.execute(cmd_create_analyzer_third_index)
session.execute(cmd_create_vector_index)
session.execute(cmd_create_vector_second_index)

In [35]:
import pandas as pd
import numpy as np
data_file = 'export_product_15000.json'
df = pd.read_json(data_file)
df.dropna()

df['brand'] = df['brand'].str.replace(r'\s+|\\n<[^<>]*>', '', regex=True)
df['product_name'] = df['product_name'].str.replace(r'\s+|\\n<[^<>]*>', '', regex=True)
df['short_description'] = df['short_description'].str.replace(r'\s+|\\n<[^<>]*>', '', regex=True)
df['long_description'] = df['long_description'].str.replace(r'\s+|\\n<[^<>]*>', '', regex=True)
df['product_categories'] = df['product_categories'].str.replace(r'\s+|\\n<[^<>]*>', '', regex=True)

df["combined_th"] = (
     df.brand.str.strip() + "." + df.product_name.str.strip() + "." + df.short_description.str.strip() + "." + df.long_description.str.strip() + "." + df.product_categories.str.strip()
)

df["combined_en"] = (
     df.product_name_en.str.strip() + "." + df.short_description_en.str.strip() + "." + df.long_description_en.str.strip()
)

products_list = df.replace(np.nan, '', regex=True)
products_list.head(5)

,product_id,brand,product_name,product_name_en,short_description,short_description_en,long_description,long_description_en,product_link,image_link,availability,sale_price,product_categories,spec,combined_th,combined_en
0,60266662,KASSA,ปุ่มกดน้ำด้านบนแบบDUALFLUSHKASSAรุ่นP2301สีโคร...,TOP DUAL FLUSH KASSA P2301 Chrome,วัสดุผลิตจากพลาสติกคุณภาพชุบโครเมียมเพิ่มความส...,วัสดุผลิตจากพลาสติกคุณภาพชุบโครเมียม เพิ่มความ...,วัสดุผลิตจากพลาสติกคุณภาพมีความแข็งแรงเหนียวทน...,วัสดุผลิตจากพลาสติกคุณภาพ มีความแข็งแรง เหนียว...,https://www.thaiwatsadu.com/th/product/ปุ่มกดน...,[https://pim.thaiwatsadu.com/TWDPIM/web/Thumbn...,True,95.0,อะไหล่สุขภัณฑ์,"[{'name': 'แบรนด์', 'value': 'KASSA'}, {'name'...",KASSA.ปุ่มกดน้ำด้านบนแบบDUALFLUSHKASSAรุ่นP230...,TOP DUAL FLUSH KASSA P2301 Chrome.วัสดุผลิตจาก...
1,60271307,GIANTKINGKONG,อะไหล่ลูกกลิ้งทาสีขนขาวGIANTKINGKONGรุ่นPR-004...,Paint Roller Accessories GIANT KINGKONG PR - 0...,ทำจากวัสดุคุณภาพช่วยให้การทาราบรื่นไม่เปรอะเปื...,ทำจากวัสดุคุณภาพ ช่วยให้การทาราบรื่น ไม่เปรอะเ...,ลูกกลิ้งสำหรับทาสีเพื่อตกแต่งผนังให้สวยงามผลิต...,ลูกกลิ้งสำหรับทาสี เพื่อตกแต่งผนังให้สวยงาม ผล...,https://www.thaiwatsadu.com/th/product/อะไหล่ล...,[https://pim.thaiwatsadu.com/TWDPIM/web/Thumbn...,True,38.0,ลูกกลิ้งทาสีและอุปกรณ์ประกอบ,"[{'name': 'ประเภท', 'value': 'ลูกกลิ้งทาสี'}, ...",GIANTKINGKONG.อะไหล่ลูกกลิ้งทาสีขนขาวGIANTKING...,Paint Roller Accessories GIANT KINGKONG PR - 0...
2,60272051,KASSAHOME,ไม้แขวนเสื้อพลาสติกKASSAHOMEรุ่นTWD1174ขนาด19....,Cloth Hanger KASSA HOME TWD1174 Size 19.2 x 40...,สำหรับแขวนเสื้อผ้าทั้งขณะเปียกและผ้าแห้งเพื่อล...,สำหรับแขวนเสื้อผ้าทั้งขณะเปียกและผ้าแห้ง เพื่อ...,ใช้สำหรับตากหรือแขวนเสื้อผ้าทั่วไปผลิตจากพลาสต...,ใช้สำหรับตาก หรือแขวนเสื้อผ้าทั่วไป ผลิตจากพลา...,https://www.thaiwatsadu.com/th/product/ไม้แขวน...,[https://pim.thaiwatsadu.com/TWDPIM/web/Thumbn...,True,48.0,ไม้แขวนผ้า/ห่วงตากผ้า/ไม้หนีบผ้า,"[{'name': 'ประเภทสินค้า', 'value': 'อุปกรณ์ทำค...",KASSAHOME.ไม้แขวนเสื้อพลาสติกKASSAHOMEรุ่นTWD1...,Cloth Hanger KASSA HOME TWD1174 Size 19.2 x 40...
3,60272054,KASSAHOME,ไม้แขวนเสื้อพลาสติกKASSAHOMEรุ่น1179ขนาด19.2x4...,Cloth Hanger KASSA HOME No. 1179 Size 19.2 x 4...,สำหรับแขวนเสื้อผ้าทั้งขณะเปียกและผ้าแห้งเพื่อล...,สำหรับแขวนเสื้อผ้าทั้งขณะเปียกและผ้าแห้ง เพื่อ...,ใช้สำหรับตากหรือแขวนเสื้อผ้าทั่วไปผลิตจากพลาสต...,ใช้สำหรับตาก หรือแขวนเสื้อผ้าทั่วไป ผลิตจากพลา...,https://www.thaiwatsadu.com/th/product/ไม้แขวน...,[https://pim.thaiwatsadu.com/TWDPIM/web/Thumbn...,True,48.0,ไม้แขวนผ้า/ห่วงตากผ้า/ไม้หนีบผ้า,"[{'name': 'ประเภทสินค้า', 'value': 'อุปกรณ์ทำค...",KASSAHOME.ไม้แขวนเสื้อพลาสติกKASSAHOMEรุ่น1179...,Cloth Hanger KASSA HOME No. 1179 Size 19.2 x 4...
4,60275968,KASSAHOTEL,ปลอกหมอนหนุนKASSAHOTELรุ่น250Tขนาด20x30+6นิ้วส...,Pillow Case KASSA HOTEL No. 250T Size 20 x 30 ...,ใช้สำหรับเปลี่ยนสลับกับหมอนหนุนผลิตจากCottonคุ...,ใช้สำหรับเปลี่ยนสลับกับหมอนหนุน ผลิตจาก Cotton...,ใช้สำหรับเปลี่ยนสลับกับหมอนหนุนผลิตจากผ้าCotto...,ใช้สำหรับเปลี่ยนสลับกับหมอนหนุน ผลิตจากผ้า Cot...,https://www.thaiwatsadu.com/th/product/ปลอกหมอ...,[https://pim.thaiwatsadu.com/TWDPIM/web/Thumbn...,True,88.0,ปลอกหมอนหนุน,"[{'name': 'ประเภทสินค้า', 'value': 'ปลอกหมอนหน...",KASSAHOTEL.ปลอกหมอนหนุนKASSAHOTELรุ่น250Tขนาด2...,Pillow Case KASSA HOTEL No. 250T Size 20 x 30 ...


In [36]:
from cassandra.query import SimpleStatement
count = 0
for index, row in products_list.iterrows():
  count += 1
  imagelink = ''.join(map(str,row.image_link))
  saleprice = str(row.sale_price)
  openai_description_embedding_th = generate_embedding(row.combined_th)
  openai_description_embedding_en = generate_embedding(row.combined_en)

  query = SimpleStatement(
                f"""
                INSERT INTO {keyspace}.products_cg_hybrid
                (product_id, brand, product_name, product_name_en, short_description,short_description_en, long_description, long_description_en, product_link, image_link, saleprice, product_categories, openai_description_embedding_th,openai_description_embedding_en )
                VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s, %s,%s, %s, %s, %s)
                """
            )
  print(f""" id: {row.product_id} and {count} records inserted.""")

  session.execute(query, (row.product_id, row.brand, row.product_name,row.product_name_en,row.short_description,row.short_description_en, row.long_description, row.long_description_en, row.product_link, imagelink, str(row.sale_price) , row.product_categories, openai_description_embedding_th, openai_description_embedding_en ))

 id: 60266662 and 1 records inserted.
 id: 60271307 and 2 records inserted.
 id: 60272051 and 3 records inserted.
 id: 60272054 and 4 records inserted.
 id: 60275968 and 5 records inserted.
 id: 60275986 and 6 records inserted.
 id: 60277313 and 7 records inserted.
 id: 60277775 and 8 records inserted.
 id: 60279268 and 9 records inserted.
 id: 60279928 and 10 records inserted.
 id: 60284279 and 11 records inserted.
 id: 60284310 and 12 records inserted.
 id: 60284733 and 13 records inserted.
 id: 60285775 and 14 records inserted.
 id: 60287659 and 15 records inserted.
 id: 60288785 and 16 records inserted.
 id: 60289006 and 17 records inserted.
 id: 60296648 and 18 records inserted.
 id: 60299370 and 19 records inserted.
 id: 60300579 and 20 records inserted.
 id: 60303106 and 21 records inserted.
 id: 60305515 and 22 records inserted.
 id: 60305749 and 23 records inserted.
 id: 60308333 and 24 records inserted.
 id: 60310829 and 25 records inserted.
 id: 60311124 and 26 records inser

In [60]:
#query english embeddings with english prompt
from cassandra.query import SimpleStatement
import pandas as pd
import numpy as np

keyword = "KASSAHOME cloth hanger white colour"
embedding = generate_embedding(keyword)

query = SimpleStatement(
    f"""
    SELECT product_id, product_name_en, short_description_en, long_description_en
    FROM {keyspace}.products_cg_hybrid
    WHERE brand : 'KASSAHOME'
    ORDER BY openai_description_embedding_en ANN OF {embedding}
    LIMIT 10 """
    )

results = session.execute(query)

top_products = results._current_rows
for row in top_products:
  print(f"""{row["product_id"]}, {row["product_name_en"]}, {row["short_description_en"]}, {row["long_description_en"]}\n""")


60327785, Cloth Hanger American KASSA HOME No.3864 Size 24 x 45 x 1.5 cm. (Pack 3 Pcs.) White - Grey, Two -tone design hanger  White frame, screwing with gray.  Made from good quality PP plastic, not easily broken. Beautiful  Lightweight, Shirts for hanging clothes. Made from good quality PP plastic, not easily broken. The head hanger is made of durable steel. Beautiful Two -tone design  White structure, gray. Lightweight Easy to move. 3 pieces/pack Width x deep x Height (24 x 45 x 1.5  cm) White - Gray

60324886, Plastic Cloth Hanger KASSA HOME LBS42 Size 41 x 1.2 x 22 CM. (Pack 3 Pcs.) Black, Good quality hanger from Kassa Home is made of quality grade plastic. The steel head is well -moistened, strong, durable, maintaining good shape. Not twisting or easily stranded.  can be used to hang both wet and dry cloth Helps to prevent wrinkled clothes And falling to the ground., Suitable for hanging clothes To prevent wrinkled clothes  and fall to the floor. Made from quality grade plastic 

In [57]:
#query thai embeddings with thai prompt
from cassandra.query import SimpleStatement
import pandas as pd
import numpy as np

keyword = "ผลิตภัณฑ์ไม้ปูพื้นที่ติดตั้งง่าย CONWOOD"
embedding = generate_embedding(keyword)

query = SimpleStatement(
    f"""
    SELECT product_id, product_name, short_description, long_description
    FROM {keyspace}.products_cg_hybrid
    WHERE brand : 'CONWOOD'
    ORDER BY openai_description_embedding_th ANN OF {embedding}
    LIMIT 10 """
    )

results = session.execute(query)

top_products = results._current_rows
for row in top_products:
  print(f"""{row["product_id"]}, {row["product_name"]}, {row["short_description"]}, {row["long_description"]}\n""")


60277577, ไม้ตกแต่งพื้นหน้า6นิ้วCONWOODรุ่นไลเนอร์ขนาด15x305x2.5ซม.สีธรรมชาติ, ตอบสนองทุกการสร้างสรรค์ด้วยไม้ตกแต่งพื้นCONWOODมอบความโดดเด่นด้วยลายเสี้ยนไม้ละเอียดสวยงามเหมือนไม้ธรรมชาติอันเป็นเอกลักษณ์ของไม้คอนวูดให้ความสวยเหมือนปูพื้นด้วยไม้จริงผลิตด้วยเทคโนโลยีล้ำสมัยมีความแข็งแกร่งทนทานทุกสภาพอากาศปลอดภัยจากปลวกสามารถใช้ติดตั้งได้ทั้งภายในและภายนอกอาคารและพื้นที่ภูมิสถาปัตย์ให้คุณรู้สึกถึงสัมผัสที่ใกล้ชิดธรรมชาติยิ่งขึ้น, ไม้ตกแต่งพื้นหน้า6นิ้วรุ่นไลเนอร์ผลิตจากปูนซีเมนต์พอร์ตแลนด์คุณภาพสูงและไฟเบอร์เซลลูโรสสูตรเฉพาะของCONWOODวัสดุปราศจากส่วนผสมของแร่ใยหินทนทานต่อสภาพอากาศไม่ลามไฟปลอดภัยจากปลวกและแมลงศัตรูกินไม้ลายเสี้ยนไม้ละเอียดสวยงามเหมือนปูพื้นด้วยไม้จริงจากธรรมชาติติดตั้งง่ายงานเสร็จไวช่วยประหยัดเวลาและต้นทุนค่าแรงใช้ได้ทั้งงานภายในภายนอกอาคารและพื้นที่ภูมิสถาปัตย์การติดตั้งต้องวางบนพื้นซีเมนต์เรียบเท่านั้นเป็นผลิตภัณฑ์สีเขียว(Eco-Friendly)ได้การรับรองจากประเทศสิงคโปร์จำนวนการใช้งาน2.18แผ่น/ตร.ม.ความหนา2.5ซม.ขนาดแผ่น(กxย)15x305ซม.น้ำหนักต่อแผ่น14.7กก.สีธรรมชาติข้อมูลทางเทคนิคข